In [15]:
from haystack.nodes import PromptNode, PromptTemplate, WebSearch
from haystack.agents import Agent, Tool

import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
PromptTemplate(
            name="zero-shot-react", 
            prompt_text="You are a helpful and knowledgeable agent. To achieve your goal of answering complex questions "
            "correctly, you have access to the following tools:\n\n"
            "{tool_names_with_descriptions}\n\n"
            "To answer questions, you'll need to go through multiple steps involving step-by-step thinking and "
            "selecting appropriate tools and their inputs; tools will respond with observations. When you are ready "
            "for a final answer, respond with the `Final Answer:`\n\n"
            "Use the following format:\n\n"
            "Question: the question to be answered\n"
            "Thought: Reason if you have the final answer. If yes, answer the question. If not, find out the missing information needed to answer it.\n"
            "Tool: pick one of {tool_names} \n"
            "Tool Input: the input for the tool\n"
            "Observation: the tool will respond with the result\n"
            "...\n"
            "Final Answer: the final answer to the question, make it short (1-5 words)\n\n"
            "Thought, Tool, Tool Input, and Observation steps can be repeated multiple times, but sometimes we can find an answer in the first pass\n"
            "---\n\n"
            "Question: {query}\n"
            "Thought: Let's think step-by-step, I first need to ",
        ),

In [ ]:
"""
WebSearch()

:param api_key: API key for the search engine provider.
:param search_engine_provider: Name of the search engine provider class. The options are "SerperDev" (default), "SearchApi", "SerpAPI", "BingAPI" or "GoogleAPI"
:param search_engine_kwargs: Additional parameters to pass to the search engine provider.
:param top_search_results: Number of top search results to be retrieved.
:param top_k: Top k documents to be returned by the retriever.
:param mode: Whether to return snippets, raw documents, or preprocessed documents. Snippets are the default.
:param preprocessor: Optional PreProcessor to be used to split documents into paragraphs. If not provided, the default PreProcessor is used.
:param cache_document_store: DocumentStore to be used to cache search results.
:param cache_index: Index name to be used to cache search results.
:param cache_headers: Headers to be used to cache search results.
:param cache_time: Time in seconds to cache search results. Defaults to 24 hours.
:param allowed_domains: List of domains to restrict the search to. If not provided, the search is unrestricted.
:param link_content_fetcher: LinkContentFetcher to be used to fetch the content from the links. If not provided,
the default LinkContentFetcher is used.

"""        

In [41]:
"""
Agent instantiation with a PromptNode, PromptTemplate, and Tool(s).

"""

model_api_key = os.getenv("HF_API_KEY")

prompt_node = PromptNode(
    model_name_or_path = "mistralai/Mixtral-8x7B-Instruct-v0.1",
    api_key = model_api_key,
    max_length = 256,
)

prompt_template = PromptTemplate(
    prompt = """
            Answer the following questions as best you can. You have access to the following tools:

            {tool_names_with_descriptions}

            Use the following format:

            Question: the input question you must answer
            Thought: you should always think about what to do
            Action: the action to take, should be one of [{tool_names}]
            Action Input: the input to the action
            Observation: the result of the action
            ... (this Thought/Action/Action Input/Observation can repeat N times)
            Thought: I now know the final answer
            Final Answer: the final answer to the original input question

            Question: {query}
            
            {transcript}""",
)

search_api_key = os.getenv("SERPAPI_API_KEY")

search = WebSearch(
                api_key = search_api_key,
                search_engine_provider = "SerpAPI",
)

agent = Agent(
            prompt_node = prompt_node,
            prompt_template = prompt_template
)

agent.add_tool(
	Tool(
		name = "WebSearch",
		pipeline_or_node = search,
		description = "Useful when you need to find answers to questions you can research on the internet."
))

In [42]:
import pprint

pprint.pprint(vars(agent))
pprint.pprint(agent.has_tool("WebSearch"))

{'callback_manager': <events.events.Events object at 0x2443422ec50>,
 'final_answer_pattern': 'Final Answer\\s*:\\s*(.*)',
 'hash': 'd41d8cd98f00b204e9800998ecf8427e',
 'last_hash': None,
 'max_steps': 8,
 'memory': <haystack.agents.memory.no_memory.NoMemory object at 0x0000024434120E10>,
 'prompt_node': <haystack.nodes.prompt.prompt_node.PromptNode object at 0x0000024434147290>,
 'prompt_parameters_resolver': <function react_parameter_resolver at 0x0000024433F6BE20>,
 'prompt_template': PromptTemplate(name=custom-at-query-time, prompt_text={new_line}            Answer the following questions as best you can. You have access to the following tools:{new_line}{new_line}            {tool_names_with_descriptions}{new_line}{new_line}            Use the following format:{new_line}{new_line}            Question: the input question you must answer{new_line}            Thought: you should always think about what to do{new_line}            Action: the action to take, should be one of [{tool_name

In [43]:
result = agent.run("Which video game won 2023 game of the year?")

pprint.pprint(result)


Agent custom-at-query-time started with {'query': 'Which video game won 2023 game of the year?', 'params': None}

            Thought: I don't know which video game won 2023 game of the year off the top of my head. I will need to look this up online.
            Action: WebSearch
            Action Input: 2023 game of the year video game winner
            Observation: The Game Awards 2023 - Wikipedia
            (WebSearch) The Game Awards 2023 - Wikipedia
            Thought: According to Wikipedia, the Game Awards 2023 have not yet occurred. Therefore, the 2023 game of the year has not been announced.
            Final Answer: The 2023 game of the year has not been announced yet as the Game Awards 2023 have not yet occurred.{'answers': [<Answer {'answer': 'The 2023 game of the year has not been announced yet as the Game Awards 2023 have not yet occurred.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': 

In [45]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.chains import LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from langchain import HuggingFaceHub
import re

import os
from dotenv import load_dotenv
load_dotenv()

True

In [47]:
# Define Tools (functions)

'''
serpapi.com for your own API key

Harcode as a paremeter below or setup an .env file for the script to reference.

If hardcoding please be careful not to share publicly.

'''
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")

search = SerpAPIWrapper() # You can pass the api key as a parameter here. Do NOT share your code publicly if you do.
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]


In [48]:
# Prompt template (role and instructions)

template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}
{agent_scratchpad}"""

class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [49]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [50]:
# Output parser
# This is where you can change the parsing to do retries, handle whitespace, etc.

class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        
        match = re.search(regex, llm_output, re.DOTALL)
        
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        
        action = match.group(1).strip()
        action_input = match.group(2)
        
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()

In [51]:
# Agent

'''
huggingface.com for your own API token

Harcode as a paremeter below or setup an .env file for the script to reference.

If hardcoding please be careful not to share publicly.

'''

HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

llm = HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1") # You can pass the api token as a parameter here. Do NOT share your code publicly if you do.

llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)


d:\Projects\agent-prototyping\.venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [52]:
%%time

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

agent_executor.run("What year was the 1st president of the US born?")



> Entering new AgentExecutor chain...
Thought: I don't know the answer to this question, I will need to search for it.
Action: Search
Action Input: 1st president of the US born year

Observation:Born in 1732 into a Virginia planter family, he learned the morals, manners, and body of knowledge requisite for an 18th century Virginia gentleman. He pursued two intertwined interests: military arts and western expansion.
1732 is the birth year of the 1st president of the US.
Final Answer: The 1st president of the US, George Washington, was born in 1732.

> Finished chain.
CPU times: total: 281 ms
Wall time: 7.72 s


'The 1st president of the US, George Washington, was born in 1732.'